In [157]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
from tensorflow import keras
import time

In [158]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [159]:
def load_img(src):
    img = cv2.imread(src)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if not img.shape == (32, 32):
        img = cv2.resize(img, (32, 32))
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    img = cv2.bitwise_not(img)
    img = img.flatten()
    img = img.astype('int32')
    return img

In [160]:
start = time.time()

char_dict = {"char": [], "matrix": []}
char_set = dict()
src = r"dataset/handwritten_letters"
for i in range(500):
    for folder in os.listdir(src):
        char_dict["char"].append(folder)
        if folder not in char_set:
            char_set[folder] = sorted_alphanumeric(os.listdir(src + '/' + folder))
        img = load_img(f"{src}/{folder}/{char_set[folder][i]}")
        char_dict["matrix"].append(img)


stop = time.time()
print(stop-start, 'seconds')

8.491028070449829 seconds


In [5]:
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
y = np.array(char_dict['char'])

y = le.fit_transform(y)
X = np.array(char_dict["matrix"])

/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
split = int(round(0.9 * len(X)))

train_X = X[:split]/255
train_y = y[:split]
test_X = X[split:]/255
test_y = y[split:]

In [7]:
train_X = train_X.astype('int32')
test_X = test_X.astype('int32')

In [8]:
values = []
for char in train_X:
    for value in char:
        if value not in values:
            values.append(value)
print('x values:', values)
values = []
for char in train_y:
    for value in char:
        if value not in values:
            values.append(value)
print('y values:', values)

x values: [0, 1]
y values: [1, 0]


In [9]:
train_X = tf.constant(train_X)
train_y = tf.constant(train_y)

test_X = tf.constant(test_X)
test_y = tf.constant(test_y)

In [10]:
train_X

<tf.Tensor: id=0, shape=(15750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [11]:
train_y

<tf.Tensor: id=1, shape=(15750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [12]:
test_X

<tf.Tensor: id=2, shape=(1750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [13]:
test_y

<tf.Tensor: id=3, shape=(1750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=1024))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(35, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
model.fit(train_X, train_y, epochs=20)

Train on 15750 samples
Epoch 1/20
15750/15750 [==============================] - 5s 322us/sample - loss: 1.3185 - accuracy: 0.6338
Epoch 2/20
15750/15750 [==============================] - 4s 284us/sample - loss: 0.5793 - accuracy: 0.8259
Epoch 3/20
15750/15750 [==============================] - 4s 281us/sample - loss: 0.3884 - accuracy: 0.8766
Epoch 4/20
15750/15750 [==============================] - 4s 285us/sample - loss: 0.2887 - accuracy: 0.9054
Epoch 5/20
15750/15750 [==============================] - 5s 287us/sample - loss: 0.2076 - accuracy: 0.9289
Epoch 6/20
15750/15750 [==============================] - 5s 292us/sample - loss: 0.1626 - accuracy: 0.9451
Epoch 7/20
15750/15750 [==============================] - 4s 284us/sample - loss: 0.1372 - accuracy: 0.9535
Epoch 8/20
15750/15750 [==============================] - 5s 291us/sample - loss: 0.1182 - accuracy: 0.9600
Epoch 9/20
15750/15750 [==============================] - 5s 288us/sample - loss: 0.0954 - accuracy: 0.9667
Epoch

In [16]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.9893333315849304% 
 Error on test data: 0.01066666841506958

Accuracy on test data: 0.833142876625061% 
 Error on test data: 0.16685712337493896


In [161]:
def get_wrong_predictions(preds):
    in_test_y = char_dict['char'][split:]
    chars = char_dict['char'][:35]
    print("{Actual letter} - {Predicted letter}")
    for i, prediction in enumerate(preds):
        prediction = list(prediction)
        actual_letter = in_test_y[i]
        predicted_letter = chars[prediction.index(max(prediction))]
        if predicted_letter != actual_letter:
            print(actual_letter, "-", predicted_letter)

In [162]:
get_wrong_predictions(pred_test)

{Actual letter} - {Predicted letter}
I - 1
N - H
G - Q
Q - G
B - G
Y - 1
9 - Q
N - R
G - S
6 - B
S - J
V - N
2 - Z
Y - 4
W - U
G - 9
S - J
Y - 4
R - A
9 - Q
S - J
D - Q
V - N
X - A
J - A
D - 0
9 - Q
G - F
S - J
5 - S
P - T
9 - Q
I - 1
G - 7
S - J
C - U
P - T
R - A
9 - Q
I - 1
G - C
T - E
K - H
2 - G
Y - T
U - V
I - 1
N - U
S - J
H - B
4 - U
B - A
L - 1
2 - Z
R - D
H - B
C - Z
D - 2
2 - 3
N - H
T - 7
D - B
Q - G
4 - H
E - Z
7 - T
I - 1
8 - E
T - R
A - I
C - 0
2 - G
9 - Q
N - H
G - A
A - Z
Q - A
L - I
9 - F
6 - B
Q - A
4 - 9
B - 6
P - G
9 - Q
0 - C
6 - B
8 - R
S - 2
A - H
C - U
D - G
L - I
I - 1
H - U
Q - G
K - W
2 - D
5 - F
P - A
W - H
U - V
9 - Q
N - H
G - S
6 - G
8 - G
A - D
4 - Q
2 - Z
U - V
I - J
8 - P
S - J
A - R
Q - 9
K - Z
9 - Q
8 - G
S - J
F - E
Y - V
P - F
W - U
N - A
G - Q
J - T
4 - U
B - D
L - H
2 - D
Y - Q
P - A
G - B
8 - Q
Q - 2
Y - F
5 - F
J - I
Q - E
2 - 3
9 - Q
8 - Q
S - J
A - R
D - J
3 - 5
P - D
9 - Q
I - 1
T - E
D - 0
V - Y
6 - G
8 - B
H - A
B - A
Y - G
G - 6
2 - S
W -

In [149]:
def predict_single_img(src):
    img = load_img
    prediction = model.predict(np.array([img,])/255)
    prediction = list(prediction[0])
    return chars[prediction.index(max(prediction))]

In [166]:
src = 'dataset/handwritten_letters/H/_1_11.jpg'
predict_single_img(src)

'H'